# make ACS intake dataframe catalogue

### required packages

In [1]:
import glob
import pathlib
import traceback
from datetime import datetime
import intake

import xarray as xr
from matplotlib import pyplot as plt

### start a local Dask client

In [2]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46723,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:35345,Total threads: 2
Dashboard: /proxy/37443/status,Memory: 9.00 GiB
Nanny: tcp://127.0.0.1:46827,


### ignore warnings

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Load individual catalogues

In [4]:
data_catalog_hist = intake.open_esm_datastore('/g/data/v14/tm4888/code/ACS/data-catalogue/catalogues/nested_test/ccam_noresm2-mm_historical_aus-10i_12km/ccam_noresm2-mm_historical_aus-10i_12km.json')
data_catalog_hist

,unique
variable,176
domain,1
host_GCM,1
run_type,1
host_ensemble,1
downscale_model,1
downscale_version,1
period,4
time_period,4
path,32960


In [5]:
data_catalog_ssp126 = intake.open_esm_datastore('/g/data/v14/tm4888/code/ACS/data-catalogue/catalogues/nested_test/ccam_noresm2-mm_ssp126_aus-10i_12km/ccam_noresm2-mm_ssp126_aus-10i_12km.json')
data_catalog_ssp126

,unique
variable,176
domain,1
host_GCM,1
run_type,1
host_ensemble,1
downscale_model,1
downscale_version,1
period,4
time_period,4
path,43757


# initialise df_cat

In [6]:
df_cat = intake.open_df_catalog(path="/g/data/v14/tm4888/code/ACS/data-catalogue/catalogues/root_df_catalog.csv", mode="w")

# add sources to df_cat

In [7]:
data_catalog_hist.name = "ccam_noresm2-mm_historical_aus-10i_12km"
model = "ccam_noresm2-mm_historical_aus-10i_12km"
variables = list(
    set(
        data_catalog_hist.df.variable.unique().astype(str)
    )
)

df_cat.add(
    data_catalog_hist,
    metadata={"model": model, "variable": variables}
)

df_cat

,model,variable
name,,
ccam_noresm2-mm_historical_aus-10i_12km,{ccam_noresm2-mm_historical_aus-10i_12km},"{ta400, zmla, wa20, zg300, ua850, snc, va20, rsus, ta700, va500, rlds, sfcWindmax, rlutcs, clh, tauv, zg400, ua300, ua100, wsgsmax, ts, psl, rlus, rsdsdir, mrfso, hus850, zg600, va850, zg1000, va1..."


In [8]:
data_catalog_ssp126.name = "ccam_noresm2-mm_ssp126_aus-10i_12km"
model = "ccam_noresm2-mm_ssp126_aus-10i_12km"
variables = list(
    set(
        data_catalog_hist.df.variable.unique().astype(str)
    )
)

df_cat.add(
    data_catalog_ssp126,
    metadata={"model": model, "variable": variables}
)

df_cat

,model,variable
name,,
ccam_noresm2-mm_historical_aus-10i_12km,{ccam_noresm2-mm_historical_aus-10i_12km},"{ta400, zmla, wa20, zg300, ua850, snc, va20, rsus, ta700, va500, rlds, sfcWindmax, rlutcs, clh, tauv, zg400, ua300, ua100, wsgsmax, ts, psl, rlus, rsdsdir, mrfso, hus850, zg600, va850, zg1000, va1..."
ccam_noresm2-mm_ssp126_aus-10i_12km,{ccam_noresm2-mm_ssp126_aus-10i_12km},"{ta400, zmla, wa20, zg300, ua850, snc, va20, rsus, ta700, va500, rlds, sfcWindmax, rlutcs, clh, tauv, zg400, ua300, ua100, wsgsmax, ts, psl, rlus, rsdsdir, mrfso, hus850, zg600, va850, zg1000, va1..."


# save the df_cat

In [9]:
df_cat.save()

# can we now make use of this df_cat?

In [20]:
df_cat = intake.open_df_catalog(
    path="/g/data/v14/tm4888/code/ACS/data-catalogue/catalogues/root_df_catalog.csv",
    columns_with_iterables=["variable"],
)

In [21]:
df_cat

,model,variable
name,,
ccam_noresm2-mm_historical_aus-10i_12km,{ccam_noresm2-mm_historical_aus-10i_12km},"{wa150, hus10, va925, ua100m, vas, wa500, ua20, CIN, prc, ta150, huss, va250, tauv, ua300, wa300, wa20, tasmin, zg50, zg700, ta500, rsut, zg70, ta700, rlut, wa925, zg300, hus500, rldscs, va1000, t..."
ccam_noresm2-mm_ssp126_aus-10i_12km,{ccam_noresm2-mm_ssp126_aus-10i_12km},"{wa150, hus10, va925, ua100m, vas, wa500, ua20, CIN, prc, ta150, huss, va250, tauv, ua300, wa300, wa20, tasmin, zg50, zg700, ta500, rsut, zg70, ta700, rlut, wa925, zg300, hus500, rldscs, va1000, t..."


In [22]:
df_cat.unique()['variable']

['wa150',
 'hus10',
 'va925',
 'ua100m',
 'vas',
 'wa500',
 'ua20',
 'CIN',
 'prc',
 'ta150',
 'huss',
 'va250',
 'tauv',
 'ua300',
 'wa300',
 'wa20',
 'tasmin',
 'zg50',
 'zg700',
 'ta500',
 'rsut',
 'zg70',
 'ta700',
 'rlut',
 'wa925',
 'zg300',
 'hus500',
 'rldscs',
 'va1000',
 'ta10',
 'hus250',
 'tasmax',
 'ta250',
 'hus1000',
 'ua50m',
 'ua400',
 'ta70',
 'hus70',
 'va300m',
 'sfcWindmax',
 'ta850',
 'va200m',
 'ua100',
 'ua200',
 'va70',
 'uas',
 'z0',
 'evspsblpot',
 'hurs',
 'mrfsol',
 'rsutcs',
 'mrfso',
 'zg150',
 'ts',
 'tauu',
 'va100',
 'va600',
 'ua250',
 'zg1000',
 'zg20',
 'rlutcs',
 'rsus',
 'zg200',
 'hus400',
 'rlds',
 'ua150m',
 'zg500',
 'mrso',
 'zg850',
 'va500',
 'va10',
 'wa850',
 'ta400',
 'wa30',
 'hfls',
 'sfcWind',
 'zg600',
 'rsdscs',
 'mrfsos',
 'clivi',
 'wa700',
 'mrro',
 'ta30',
 'ua850',
 'va150m',
 'ta20',
 'mrsos',
 'ua700',
 'ps',
 'hus200',
 'snd',
 'zmla',
 'evspsbl',
 'snw',
 'rluscs',
 'va150',
 'wa250',
 'ta100',
 'ta300',
 'clwvi',
 'ua300m'

# find all variables with "tas" via `regex`

In [32]:
tas_cat = df_cat.search(variable=".*tas.*")
tas_cat

,model,variable
name,,
ccam_noresm2-mm_historical_aus-10i_12km,{ccam_noresm2-mm_historical_aus-10i_12km},"{tas, tasmin, tasmax}"
ccam_noresm2-mm_ssp126_aus-10i_12km,{ccam_noresm2-mm_ssp126_aus-10i_12km},"{tas, tasmin, tasmax}"


In [33]:
tas_hist_cat = df_cat.search(variable="tas",model="ccam_noresm2-mm_historical_aus-10i_12km")
tas_hist_cat

,model,variable
name,,
ccam_noresm2-mm_historical_aus-10i_12km,{ccam_noresm2-mm_historical_aus-10i_12km},{tas}


In [34]:
tas_hist_cat.df

,model,variable,name,yaml
0,ccam_noresm2-mm_historical_aus-10i_12km,[tas],ccam_noresm2-mm_historical_aus-10i_12km,sources:\n ccam_noresm2-mm_historical_aus-10i_12km:\n args:\n obj: /g/data/v14/tm4888/code/ACS/data-catalogue/catalogues/nested_test/ccam_noresm2-mm_historical_aus-10i_12km/ccam_noresm2-m...


# Load the `intake` source for your search

In [37]:
tas_hist_source = tasmax_hist_cat.to_source()
tas_hist_source

,unique
variable,176
domain,1
host_GCM,1
run_type,1
...,...
period,4
time_period,4
path,32960
derived_variable,0


# Developer (Dougie) working on way to pass higher level search onto the source

# for now must search twice

In [42]:
tas_hist_source = tas_hist_source.search(variable="tas")
tas_hist_source

,unique
variable,1
domain,1
host_GCM,1
run_type,1
...,...
period,3
time_period,3
path,192
derived_variable,0


# use `keys_info()`

In [43]:
tas_hist_source.keys_info()

,domain,host_GCM,run_type,host_ensemble,downscale_model,downscale_version,period
key,,,,,,,
AUS-10i.NCC-NorESM2-MM.historical.r1i1p1f1.CSIRO-CCAM-2203.v1.1hr,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr
AUS-10i.NCC-NorESM2-MM.historical.r1i1p1f1.CSIRO-CCAM-2203.v1.day,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,day
AUS-10i.NCC-NorESM2-MM.historical.r1i1p1f1.CSIRO-CCAM-2203.v1.mon,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon


# select only monthly data

In [45]:
monthly_tas_hist = tas_hist_source.search(period='mon')
monthly_tas_hist

,unique
variable,1
domain,1
host_GCM,1
run_type,1
...,...
period,1
time_period,1
path,64
derived_variable,0


In [46]:
monthly_tas_hist.df

,variable,domain,host_GCM,run_type,host_ensemble,downscale_model,downscale_version,period,time_period,path
0,tas,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/mon/tas/tas_AUS-10i_NCC-NorESM2-MM_historic...
1,tas,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/mon/tas/tas_AUS-10i_NCC-NorESM2-MM_historic...
2,tas,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/mon/tas/tas_AUS-10i_NCC-NorESM2-MM_historic...
3,tas,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/mon/tas/tas_AUS-10i_NCC-NorESM2-MM_historic...
...,...,...,...,...,...,...,...,...,...,...
60,tas,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/mon/tas/tas_AUS-10i_NCC-NorESM2-MM_historic...
61,tas,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/mon/tas/tas_AUS-10i_NCC-NorESM2-MM_historic...
62,tas,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/mon/tas/tas_AUS-10i_NCC-NorESM2-MM_historic...
63,tas,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/mon/tas/tas_AUS-10i_NCC-NorESM2-MM_historic...


# load catalogue entries into a dictionary of `xarray` datasets

In [49]:
DS_dict = monthly_tas_hist.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'domain.host_GCM.run_type.host_ensemble.downscale_model.downscale_version.period'


HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 494 in H5O__attr_open_by_name(): can't locate attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Object not found
HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Ao

In [50]:
DS_dict

{'AUS-10i.NCC-NorESM2-MM.historical.r1i1p1f1.CSIRO-CCAM-2203.v1.mon': <xarray.Dataset>
 Dimensions:   (time: 768, lat: 611, lon: 928, bnds: 2)
 Coordinates:
   * time      (time) object 1951-01-15 12:00:00 ... 2014-12-15 12:00:00
     lat_bnds  (lat, bnds) float32 dask.array<chunksize=(611, 2), meta=np.ndarray>
     lon_bnds  (lon, bnds) float32 dask.array<chunksize=(928, 2), meta=np.ndarray>
   * lon       (lon) float64 89.3 89.4 89.5 89.6 89.7 ... 181.7 181.8 181.9 182.0
   * lat       (lat) float64 -52.3 -52.2 -52.1 -52.0 -51.9 ... 8.4 8.5 8.6 8.7
 Dimensions without coordinates: bnds
 Data variables:
     tas       (time, lat, lon) float32 dask.array<chunksize=(12, 611, 928), meta=np.ndarray>
 Attributes: (12/29)
     axiom_version:                       0.1.0
     axiom_schemas_version:               0.1.0
     axiom_schema:                        CORDEX
     contact:                             Marcus Thatcher (Marcus.Thatcher@csi...
     Conventions:                         CF-1

In [51]:
DS = monthly_tas_hist.to_dask()

HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 494 in H5O__attr_open_by_name(): can't locate attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Object not found
HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Ao

In [52]:
DS

<xarray.Dataset>
Dimensions:   (time: 768, lat: 611, lon: 928, bnds: 2)
Coordinates:
  * time      (time) object 1951-01-15 12:00:00 ... 2014-12-15 12:00:00
    lat_bnds  (lat, bnds) float32 dask.array<chunksize=(611, 2), meta=np.ndarray>
    lon_bnds  (lon, bnds) float32 dask.array<chunksize=(928, 2), meta=np.ndarray>
  * lon       (lon) float64 89.3 89.4 89.5 89.6 89.7 ... 181.7 181.8 181.9 182.0
  * lat       (lat) float64 -52.3 -52.2 -52.1 -52.0 -51.9 ... 8.4 8.5 8.6 8.7
Dimensions without coordinates: bnds
Data variables:
    tas       (time, lat, lon) float32 dask.array<chunksize=(12, 611, 928), meta=np.ndarray>
Attributes: (12/29)
    axiom_version:                       0.1.0
    axiom_schemas_version:               0.1.0
    axiom_schema:                        CORDEX
    contact:                             Marcus Thatcher (Marcus.Thatcher@csi...
    Conventions:                         CF-1.7
    CORDEX_domain:                       AUS-10i
    ...                                  ...
    intake_esm_attrs:downscale_model:    CSIRO-CCAM-2203
    intake_esm_attrs:downscale_version:  v1
    intake_esm_attrs:period:             mon
    intake_esm_attrs:time_period:        monthly
    intake_esm_attrs:_data_format_:      netcdf
    intake_esm_dataset_key:              AUS-10i.NCC-NorESM2-MM.historical.r1...

In [17]:
%%time
DS_dict = tas_hist_cat.to_source().to_dataset_dict()
DS_dict


--> The keys in the returned dictionary of datasets are constructed as follows:
	'domain.host_GCM.run_type.host_ensemble.downscale_model.downscale_version.period'


Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2023-05-15 14:56:08,396 - distributed.nanny - ERROR - Worker process died unexpectedly
Traceback (most recent call last):
  File "/g/data/v14/tm4888/miniconda3/envs/intake-stable-ecgtools/lib/python3.10/site-packages/distributed/nanny.py", line 986, in _run
    asyncio.run(run())
  File "/g/data/v14/tm4888/miniconda3/envs/intake-stable-ecgtools/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/g/data/v14/tm4888/miniconda3/envs/intake-stable-ecgtools/lib/python3.10/asyncio/base_events.py", line 636, in run_until_complete
    self.run_forever()
  File "/g/data/v14/tm4888/miniconda3/envs/intake-stable-ecgtools/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/g/data/v14/tm4888/miniconda3/envs/intake-stable-ecgtools/lib/python3.1

In [ ]:
tas_cat_source_dict = tas_cat.to_source_dict()
tas_cat_source_dict

In [ ]:
tas_cat_source_dict['ccam_noresm2-mm_historical_aus-10i_12km'].to_dataset_dict()

# load monthly surface air temperature for both historical and ssp126

In [ ]:
TAS_month_hist = data_catalog_hist.search(variable=['tas'],period='mon')
TAS_month_ssp126 = data_catalog_ssp126.search(variable=['tas'],period='mon')

In [ ]:
DS_dict_hist = TAS_month_hist.to_dataset_dict()

In [ ]:
DS_dict_ssp126 = TAS_month_ssp126.to_dataset_dict()

In [ ]:
ssp126_keys = list(DS_dict_ssp126.keys())
hist_keys = list(DS_dict_hist.keys())

In [ ]:
ssp126_keys

In [ ]:
DS_ssp126 = DS_dict_ssp126[ssp126_keys[0]]
DS_hist = DS_dict_hist[hist_keys[0]]

In [ ]:
DS_hist

# do some basic comparisons

### Time mean TAS SSP126 - Historical

In [ ]:
hist_tas_mean = DS_hist.tas.mean('time')
ssp126_tas_mean = DS_ssp126.tas.mean('time')                         

In [ ]:
diff_tas_mean = ssp126_tas_mean - hist_tas_mean
diff_tas_mean = diff_tas_mean.compute()

In [ ]:
diff_tas_mean.plot()
plt.title('Time mean TAS SSP126 - Historical')


### climatology

In [ ]:
%%time
hist_season = DS_hist.tas.groupby('time.season').mean('time').compute()
ssp126_season = DS_ssp126.tas.groupby('time.season').mean('time').compute()

In [ ]:
DJF_diff = ssp126_season.sel(season='DJF') - hist_season.sel(season='DJF')
DJF_diff = DJF_diff.compute()

In [ ]:
DJF_diff.plot()
plt.title('DJF season : SSP126 - Historical')

# can we load from the single nested catalogue?

In [ ]:
root_cat= intake.open_catalog('/g/data/v14/tm4888/code/ACS/data-catalogue/catalogues/nested_test/ACS_root_catalogue.yaml')

In [ ]:
root_cat

In [ ]:
list(root_cat)

In [ ]:
print(dir(root_cat))

In [ ]:
TAS_month_hist = root_cat.ccam_noresm2_mm_historical_aus_10i_12km.search(variable=['tas'],period='mon')
TAS_month_ssp126 = root_cat.ccam_noresm2_mm_ssp126_aus_10i_12km.search(variable=['tas'],period='mon')

In [ ]:
DS_dict_hist = TAS_month_hist.to_dataset_dict()
DS_dict_ssp126 = TAS_month_ssp126.to_dataset_dict()
ssp126_keys = list(DS_dict_ssp126.keys())
hist_keys = list(DS_dict_hist.keys())
DS_ssp126 = DS_dict_ssp126[ssp126_keys[0]]
DS_hist = DS_dict_hist[hist_keys[0]]

In [ ]:
hist_keys

In [ ]:
TAS_month_ssp126.df

In [ ]:
TAS_month_ssp126.df.variable.unique()

In [ ]:
TAS_month_ssp126.df.period.unique()

In [ ]:
mean_tas = DS_hist.tas.mean(dim='time')
mean_tas

In [ ]:
%%time
mean_tas = mean_tas.compute()

In [ ]:
mean_temp = mean_tas - 273.15

In [ ]:
mean_temp.plot(robust=True)
plt.title('Mean Near-Surface Air Temperature in degrees C for \n AUS-10i.NCC-NorESM2-MM.historical.r1i1p1f1.CSIRO-CCAM-2203.v1.mon')

# seasonal climatology

In [ ]:
tas_season_mean = DS_hist.tas.groupby('time.season').mean('time')

In [ ]:
tas_season_mean

In [ ]:
temp_season_mean = tas_season_mean-273.15

In [ ]:
temp_season_mean

In [ ]:
temp_season_mean.sel(season='JJA').plot(robust=True)
plt.title('Winter(JJA) Near-Surface Air Temperature in degrees C for \n AUS-10i.NCC-NorESM2-MM.historical.r1i1p1f1.CSIRO-CCAM-2203.v1.mon')

# how can we more elegantly pull data out of cats?

### seems like we can't search across all data sources at once?

In [ ]:
root_cat.search(variable=['tas'])

### show user all data sources

In [ ]:
def list_data_sources(root_cat=root_cat):
    data_sources_list = list(root_cat)
    print('all data source keys are:')
    print(data_sources_list)
    return data_sources_list

In [ ]:
data_sources_list = list_data_sources()

### we want to load datasets with data from a single source per search

#### How to search across key string? "find all noresm2 based runs"

In [ ]:
search_string = 'noresm2'
indicies = [i for i, s in enumerate(data_sources_list) if search_string in s]
print(indicies)

In [ ]:
def search_data_source_by_string(search_string,root_cat=root_cat):
    print('search string is:'+ search_string +'\n')
    data_sources_list = list_data_sources(root_cat)
    indicies = [i for i, s in enumerate(data_sources_list) if search_string in s]
    print('\n matching indicies are:')
    for item in [str(i) for i in indicies]:
        print(item)
    matching_keys = [data_sources_list[i] for i in indicies]
    print('\n matching keys are:')
    for item in matching_keys:
        print(item)
    return indicies, matching_keys

In [ ]:
indicies, matching_keys = search_data_source_by_string('noresm2')

## pull out the DF's for a data source

In [ ]:
DF = root_cat[matching_keys[0]].df
DF

## setup searches

In [ ]:
search_dict = {}
i = 1
for key in matching_keys:
    var_name = "search" + str(i)
    search_dict[var_name] = root_cat[key].search(variable=['tas','CAPE','zmla'],time_period=['monthly','daily'])
    i +=1

In [ ]:
search_dict

In [ ]:
search_dict['search1'].unique()

In [ ]:
def search_for(root_cat=root_cat,
               variables=['tas'],
               product='ccam_noresm2_mm_historical_aus_10i_12km',
               domains=['AUS-10i'],
               host_GCMs=['NCC-NorESM2-MM'],
               run_types=['historical'],
               host_ensembles=['r1i1p1f1'],
               downscale_models = ['CSIRO-CCAM-2203'],
               downscale_versions = ['v1'],
               time_periods=['monthly']):
    search_result = root_cat[product].search(variable=variables,time_period=time_periods)
    return search_result

In [ ]:
variables = ['tas','CAPE','zmla']

In [ ]:
search_result = search_for(variables=variables)

In [ ]:
DS_dict = search_result.to_dataset_dict()

In [ ]:
DS_dict['AUS-10i.NCC-NorESM2-MM.historical.r1i1p1f1.CSIRO-CCAM-2203.v1.mon']

In [ ]:
DS = search_result.to_dask()

In [ ]:
DS.nbytes/1e9

In [ ]:
DS

In [ ]:
product='ccam_noresm2_mm_historical_aus_10i_12km'
domains=['AUS-10i']
host_GCMs=['NCC-NorESM2-MM']
run_types=['historical']
host_ensembles=['r1i1p1f1']
downscale_models = ['CSIRO-CCAM-2203']
downscale_versions = ['v1']
time_periods=['monthly']

In [ ]:
product

In [ ]:
root_cat[product]

In [ ]:
root_cat.ccam_noresm2_mm_historical_aus_10i_12km

In [ ]:
root_cat.ccam_noresm2_mm_historical_aus_10i_12km.df

In [ ]:
client.restart()

In [22]:
client.close()